In [49]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Ejercicio Uno

In [52]:
## Importar los csv airportList.csv y airportsLocation.csv
df_list=spark.read.csv("EjercicioSpark/airportList.csv",inferSchema=True, header=True)
df_loc=spark.read.csv("EjercicioSpark/airportsLocation.csv",inferSchema=True, header=True)
print("El primer DataFrame tiene",df_list.count(),"filas")
print("El segundo DataFrame tiene",df_loc.count(),"filas")

El primer DataFrame tiene 8107 filas
El segundo DataFrame tiene 8107 filas


In [53]:
#Eliminar filas con datos faltantes
df_list=df_list.dropna()
df_loc=df_loc.dropna()
print("El primer DataFrame sin datos faltantes tiene",df_list.count(),"filas")
print("El segundo DataFrame sin datos faltantes tiene",df_loc.count(),"filas")

El primer DataFrame sin datos faltantes tiene 5880 filas
El segundo DataFrame sin datos faltantes tiene 5880 filas


In [54]:
#Hacer Inner Join 
df_inner=df1.join(df2, df1["airport_code"]==df2["airport_code"]).drop(df2.airport_code)
df_inner.printSchema()

root
 |-- airport_id: integer (nullable = true)
 |-- airport_name: string (nullable = true)
 |-- city_airport_location: string (nullable = true)
 |-- country: string (nullable = true)
 |-- airport_code: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- altitude: integer (nullable = true)
 |-- timezone: double (nullable = true)
 |-- dst: string (nullable = true)
 |-- olson_timezone: string (nullable = true)



In [56]:
#Quitar columnas Innecesarias y ver esquema de datos
df_inner=df_inner.drop("olson_timezone","dst","timezone")
df_inner.printSchema()

root
 |-- airport_id: integer (nullable = true)
 |-- airport_name: string (nullable = true)
 |-- city_airport_location: string (nullable = true)
 |-- country: string (nullable = true)
 |-- airport_code: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- altitude: integer (nullable = true)



In [57]:
#Consulta de datos sobre latitud igual o mayor a 40
print("Posiblemente hay",df_inner.count(),"aeropuertos corruptos.","\nCon la siguiente informacion:")
df_inner.filter(df_inner.latitude>=40).show()

Posiblemente hay 5884 aeropuertos corruptos. 
Con la siguiente informacion:
+----------+--------------------+---------------------+---------+------------+---------+----------+--------+
|airport_id|        airport_name|city_airport_location|  country|airport_code| latitude| longitude|altitude|
+----------+--------------------+---------------------+---------+------------+---------+----------+--------+
|         7|          Narsarsuaq|         Narssarssuaq|Greenland|         UAK|61.160517|-45.425978|     112|
|         8|                Nuuk|             Godthaab|Greenland|         GOH|64.190922|-51.678064|     283|
|         9|   Sondre Stromfjord|          Sondrestrom|Greenland|         SFJ|67.016969|-50.689325|     165|
|        10|      Thule Air Base|                Thule|Greenland|         THU|76.531203|-68.703161|     251|
|        11|            Akureyri|             Akureyri|  Iceland|         AEY|65.659994|-18.072703|       6|
|        12|         Egilsstadir|          Egilsstad

Ejercicio Dos

In [60]:
#Crear una tabla temporal del dataframe df_list para poder hacer consultas SQL usando Spark
#Despues hacer un conteo de registros y finalmente imprimir los registros
df_list.createOrReplaceTempView("t")
c=spark.sql("  select * from t where (country=='Mexico') or (country=='Brazil') or (country=='United States')  "
          "  or (country=='Canada') or (country=='Japan') or (mod(airport_id,2)=1)  ").count()
print("En",c,"aeropuertos pueden existir atentados.","\nLa informacion es la siguiente:")

spark.sql("  select * from t where (country=='Mexico') or (country=='Brazil') or (country=='United States')  "
          "  or (country=='Canada') or (country=='Japan') or (mod(airport_id,2)=1)  ").show()

En 3987 aeropuertos pueden existir atentados. 
La informacion es la siguiente:
+----------+--------------------+---------------------+----------------+------------+
|airport_id|        airport_name|city_airport_location|         country|airport_code|
+----------+--------------------+---------------------+----------------+------------+
|         1|              Goroka|               Goroka|Papua New Guinea|         GKA|
|         3|         Mount Hagen|          Mount Hagen|Papua New Guinea|         HGU|
|         5|Port Moresby Jack...|         Port Moresby|Papua New Guinea|         POM|
|         7|          Narsarsuaq|         Narssarssuaq|       Greenland|         UAK|
|         9|   Sondre Stromfjord|          Sondrestrom|       Greenland|         SFJ|
|        11|            Akureyri|             Akureyri|         Iceland|         AEY|
|        13|        Hornafjordur|                 Hofn|         Iceland|         HFN|
|        15|          Isafjordur|           Isafjordur|      